<a href="https://colab.research.google.com/github/nwon24/nlp/blob/main/W5/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import os
os.environ["KERAS_BACKEND"] = "torch"
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import InputLayer
from keras.preprocessing import sequence
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn import model_selection
import torch
import torch.nn as nn

data=pd.read_csv("xaa",encoding="utf-8")

vect  = CountVectorizer()
#corpus = vect.fit_transform(data["text"])
#train_corpus, test_corpus, train_label, test_label = model_selection.train_test_split(data["text"],data["label"],test_size=0.4)
#Encoder = LabelEncoder()
#train_label = Encoder.fit_transform(train_label)
#test_label = Encoder.fit_transform(test_label)
#train_corpus_vect=vect.transform(train_corpus)
#test_corpus_vect=vect.transform(test_corpus)

#import spacy
#nlp = spacy.load("en_core_web_sm")
#for doc in nlp.pipe(data["text"],disable=["tagger","parser","ner","textcat"]):
#    # Do something with the doc here
#    print(doc.text)

corpus = vect.fit_transform(data["text"])

maxlen=0

def transform(text,vect):
    global maxlen
    d=vect.vocabulary_
    p=vect.build_preprocessor()
    t=vect.build_tokenizer()
    vec_list=[]
    for doc in text:
        tokens=t(p(doc))
        s=len(tokens)
        if s>maxlen:
            maxlen=s
        doc_vec=np.array([d[token] for token in tokens])
        #doc_vec=sequence.pad_sequences(doc_vec,maxlen=maxlen)
        vec_list.append(doc_vec)
    vec_list=sequence.pad_sequences(vec_list,maxlen=maxlen)
    corpus_vec=np.vstack(vec_list)
    return torch.tensor(corpus_vec).float()

# print(corpus_vec)

bsize=32
epochs=8
lr=1e-5

class lstm(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(lstm,self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.lstm=nn.LSTM(input_size,hidden_size)
        self.fc=nn.Linear(self.hidden_size,1)

    def forward(self,x):
        out, (_,_) = self.lstm(x)
        return nn.Sigmoid()(self.fc(out))

Encoder = LabelEncoder()
corpus_vec = transform(data["text"],vect)
train_corpus_vec, test_corpus_vec, train_label, test_label = model_selection.train_test_split(corpus_vec,data["label"],test_size=0.4)
train_label = Encoder.fit_transform(train_label)
test_label = Encoder.fit_transform(test_label)
#train_corpus_vec = transform(train_corpus,vect)
#test_corpus_vec = transform(test_corpus,vect)
print(maxlen)
print(len(train_corpus_vec[0]))
lstm_classifier=lstm(1,100)
loss_fn=nn.MSELoss()
optimizer=torch.optim.Adam(lstm_classifier.parameters(),lr=lr)
print(len(lstm_classifier(torch.reshape(train_corpus_vec[0],(maxlen,1,1)))))

#print(train_corpus_vect)
#features=train_corpus_vect.shape[1]

#model=Sequential([InputLayer(shape=(features,1)),
#                  LSTM(128),
#                  Dense(1)])
#model.compile(loss='mean_squared_error', optimizer='adam')
#model.fit(train_corpus_vect,train_label,epochs=epochs,batch_size=bsize)

191
191
191
